In [1]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [2]:
vocab_size=10000
(x_train,y_train),(x_test, y_test) = datasets.imdb.load_data(num_words = vocab_size)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [3]:
max_len = 500
x_train = pad_sequences(x_train, maxlen = max_len, padding='post')
x_test = pad_sequences(x_test, maxlen = max_len, padding='post')

In [4]:
x_train

array([[   1,   14,   22, ...,    0,    0,    0],
       [   1,  194, 1153, ...,    0,    0,    0],
       [   1,   14,   47, ...,    0,    0,    0],
       ...,
       [   1,   11,    6, ...,    0,    0,    0],
       [   1, 1446, 7079, ...,    0,    0,    0],
       [   1,   17,    6, ...,    0,    0,    0]], dtype=int32)

In [5]:
x_test

array([[   1,  591,  202, ...,    0,    0,    0],
       [   1,   14,   22, ...,    0,    0,    0],
       [  33,    6,   58, ...,    9,   57,  975],
       ...,
       [   1,   13, 1408, ...,    0,    0,    0],
       [   1,   11,  119, ...,    0,    0,    0],
       [   1,    6,   52, ...,    0,    0,    0]], dtype=int32)

In [6]:
model=Sequential()
model.add(Embedding(vocab_size, 256))
model.add(Dropout(0.2))
model.add(Conv1D(256,3, padding='valid',activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [7]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
mc = ModelCheckpoint('CNN_postmodel.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only = True)

In [8]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(x_train, y_train, epochs = 10, validation_data = (x_test, y_test), callbacks=[es, mc])

Epoch 1/10
782/782 [==============================] - 70s 49ms/step - loss: 0.3630 - acc: 0.8261 - val_loss: 0.2583 - val_acc: 0.8938

Epoch 00001: val_acc improved from -inf to 0.89380, saving model to CNN_postmodel.h5
Epoch 2/10
782/782 [==============================] - 37s 48ms/step - loss: 0.1691 - acc: 0.9367 - val_loss: 0.2598 - val_acc: 0.8952

Epoch 00002: val_acc improved from 0.89380 to 0.89524, saving model to CNN_postmodel.h5
Epoch 3/10
782/782 [==============================] - 37s 48ms/step - loss: 0.0652 - acc: 0.9783 - val_loss: 0.3470 - val_acc: 0.8868

Epoch 00003: val_acc did not improve from 0.89524
Epoch 4/10
782/782 [==============================] - 37s 48ms/step - loss: 0.0288 - acc: 0.9900 - val_loss: 0.3915 - val_acc: 0.8960

Epoch 00004: val_acc improved from 0.89524 to 0.89604, saving model to CNN_postmodel.h5
Epoch 00004: early stopping


In [9]:
loaded_model = load_model('CNN_postmodel.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

782/782 [==============================] - 6s 7ms/step - loss: 0.3915 - acc: 0.8960

 테스트 정확도: 0.8960
